# 1. Sentiment analysis of online reviews.

In [206]:
import numpy as np
import pandas as pd
from matplotlib import pylab as plt
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix
import matplotlib.cm as cm
from sklearn import metrics
from sklearn.decomposition import PCA
%matplotlib inline
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn import preprocessing
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LogisticRegression
import re
import string

(a) Download Sentiment Labelled Sentences Data Set. There are three data files under the root folder. yelp_labelled.txt, amazon_cells_labelled.txt and imdb_labelled.txt. Parse each file with the specifications in readme.txt. Are the labels balanced? If not, what’s the ratio between the two labels? Explain how you process these files.

In [59]:
#load dataset
#import pandas as pd
yelp = pd.read_csv("yelp_labelled.txt", header = None, delimiter = "\t")
amazon = pd.read_csv("amazon_cells_labelled.txt", header = None, delimiter = '\t')
imdb = pd.read_csv("imdb_labelled.txt", header = None, delimiter = '\t+')

print(yelp.shape)
print(amazon.shape)
print(imdb.shape)

#print(yelp)
#print(amazon)
#print(imdb)


(1000, 2)
(1000, 2)
(1000, 2)


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


In [55]:
yelp_value = yelp.values
yelp_1 = np.sum(yelp_value[:,1] == 1)
print("The number of 1 in yelp is ", yelp_1)
yelp_0 = np.sum(yelp_value[:,1] == 0)
print("The number of 0 in yelp is ", yelp_0)

if (yelp_1 == yelp_0):
    print("Thus, the labels in yelp are balenced")

The number of 1 in yelp is  500
The number of 0 in yelp is  500
Thus, the labels in yelp are balenced


In [18]:
amazon_value = amazon.values
amazon_1 = np.sum(amazon_value[:,1] == 1)
print("The number of 1 in amazon is ", amazon_1)
amazon_0 = np.sum(amazon_value[:,1] == 0)
print("The number of 0 in amazon is ", amazon_0)

if (amazon_1 == amazon_0):
    print("Thus, the labels in amazon are balenced")

The number of 1 in amazon is  500
The number of 0 in amazon is  500
Thus, the labels in amazon are balenced


In [60]:
imdb_value = imdb.values
imdb_1 = np.sum(imdb_value[:,1] == 1)
print("The number of 1 in imdb is ", imdb_1)
imdb_0 = np.sum(imdb_value[:,1] == 0)
print("The number of 0 in imdb is ", imdb_0)

if (imdb_1 == imdb_0):
    print("Thus, the labels in imdb are balenced")

The number of 1 in imdb is  500
The number of 0 in imdb is  500
Thus, the labels in imdb are balenced


We use pd.read_csv to read the data file and use numpy.sum to count the positive and negative amount of labels in each file. The labels are balanced since each of them has 500 positive and 500 negative in the file.

(b) Pick your preprocessing strategy. Since these sentences are online reviews, they may contain
significant amounts of noise and garbage. You may or may not want to do one or all of
the following. Explain the reasons for each of your decision (why or why not).
• Lowercase all of the words.
• Lemmatization of all the words (i.e., convert every word to its root so that all of “running,”
“run,” and “runs” are converted to “run” and and all of “good,” “well,” “better,” and “best”
are converted to “good”; this is easily done using nltk.stem).
• Strip punctuation.
• Strip the stop words, e.g., “the”, “and”, “or”.
• Something else? Tell us about it.

In [138]:
def load_data(data):
    positive_data, positive_label = [], []
    negative_data, negative_label = [], []
    f = open(data, 'r')
    for line in f:
        temp = line.strip().split('\t')
        if temp[1] == '0':
            positive_data.append(temp[0])
            positive_label.append(int(temp[1]))
        else: 
            negative_data.append(temp[0])
            negative_label.append(int(temp[1]))
    f.close()
    return positive_data, positive_label, negative_data, negative_label

amazon_positive_data, amazon_positive_label, amazon_negative_data, amazon_negative_label = read_data('amazon_cells_labelled.txt')
imdb_positive_data, imdb_positive_label, imdb_negative_data, imdb_negative_label = read_data('imdb_labelled.txt')
yelp_positive_data, yelp_positive_label, yelp_negative_data, yelp_negative_label = read_data('yelp_labelled.txt')

In [139]:
def preprocess_data(data):
    # Lowercase all of the words
    data = np.char.lower(data)
    
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    #Strip puncturation
    for i in range(len(data)):
        data[i] = re.sub(r'[^\w\s]','',data[i])

    #Lemmatization of all the words + Strip the stop words
    for i in range(len(data)):
        list_ = nltk.word_tokenize(data[i])
        sen = []
        for n in list_:
            if not n in stop_words and not n.isdigit():
                sen.append(lemmatizer.lemmatize(n))
        data[i] = ' '.join(sen)
    return data.tolist()

#processing the data
amazon_data_processed = preprocess_data(amazon_positive_data) + preprocess_data(amazon_negative_data)
imdb_data_processed = preprocess_data(imdb_positive_data) + preprocess_data(imdb_negative_data)
yelp_data_processed = preprocess_data(yelp_positive_data) + preprocess_data(yelp_negative_data)

#double check the length of the data
print(len(amazon_data_processed))
print(len(yelp_data_processed))
print(len(imdb_data_processed))

1000
1000
1000


1.Lowercase all of the words: Yes, it will help with key word matching.
2.Lemmatization of all the words: Yes, it will help to increase the accuracy.
3.Strip punctuation: Yes, punctuation is not taken into account.
4.Strip the stop words: Yes, removing stop words can help extract more important information.

(c) Split training and testing set. In this assignment, for each file, please use the first 400 instances
for each label as the training set and the remaining 100 instances as testing set. In
total, there are 2400 reviews for training and 600 reviews for testing.

In [140]:
#Split training and testing set
#use the first 400 instances for each label as the training set and the remaining 100 instances as testing set

#split training and testing set for yelp
x_Train_yelp = yelp_data_processed[0:400] + yelp_data_processed[500:900]
x_Test_yelp = yelp_data_processed[400:500] + yelp_data_processed[900:1000]
y_Train_yelp = yelp_label_processed[0:400] + yelp_label_processed[500:900]
y_Test_yelp = yelp_label_processed[400:500] + yelp_label_processed[900:1000]

#split training and testing set for amazon
x_Train_amazon = amazon_data_processed[0:400] + amazon_data_processed[500:900]
x_Test_amazon = amazon_data_processed[400:500] + yelp_data_processed[900:1000]
y_Train_amazon = amazon_label_processed[0:400] + amazon_label_processed[500:900]
y_Test_amazon = amazon_label_processed[400:500] + yelp_label_processed[900:1000]

#split training and testing set for imdb
x_Train_imdb = imdb_data_processed[0:400] + imdb_data_processed[500:900]
x_Test_imdb = imdb_data_processed[400:500] + yelp_data_processed[900:1000]
y_Train_imdb = imdb_label_processed[0:400] + imdb_label_processed[500:900]
y_Test_imdb = imdb_label_processed[400:500] + yelp_label_processed[900:1000]

#In total, there are 2400 reviews for training and 600 reviews for testing
x_train = x_Train_yelp + x_Train_amazon + x_Train_imdb
x_test = x_Test_yelp + x_Test_amazon + x_Test_imdb
y_train = y_Train_yelp + y_Train_amazon + y_Train_imdb
y_test = y_Test_yelp + y_Test_amazon + y_Test_imdb

print(len(x_train))
print(len(x_test))
print(len(y_train))
print(len(y_test))

2400
600
2400
600


In [178]:
x_train = amazon_data_positive[:400] + amazon_data_negative[:400] +imdb_data_positive[:400]+imdb_data_negative[:400]+yelp_data_positive[:400]+yelp_data_negative[:400]
x_test = amazon_data_positive[400:] + amazon_data_negative[400:] +imdb_data_positive[400:]+imdb_data_negative[400:]+yelp_data_positive[400:]+yelp_data_negative[400:]
y_train = amazon_label_positive[:400] + amazon_label_negative[:400] +imdb_label_positive[:400]+imdb_label_negative[:400]+yelp_label_positive[:400]+yelp_label_negative[:400]
y_test = amazon_label_positive[400:] + amazon_label_negative[400:] +imdb_label_positive[400:]+imdb_label_negative[400:]+yelp_label_positive[400:]+yelp_label_negative[400:]
print(len(y_train))
print(len(y_test))

2400
600


(d) Bag of Words model. Extract features and then represent each review using bag of words
model, i.e., every word in the review becomes its own element in a feature vector. In order to
do this, first, make one pass through all the reviews in the training set (Explain why we can’t
use testing set at this point) and build a dictionary of unique words. Then,make another pass
through the review in both the training set and testing set and count up the occurrences of
each word in your dictionary. The i th element of a review’s feature vector is the number of
occurrences of the i th dictionary word in the review. Implement the bag of words model and
report feature vectors of any two reviews in the training set.

In [148]:
#bag of words model.
dict_unique_words = dict()
count_unique_word = 0
for x in x_train:
    for word in x.split(' '):
        if word not in dict_unique_words:
            dict_unique_words[word] = count_unique_word
            count_unique_word +=1
train_featured = [] 
test_featured = []  
for x in x_train:
    word_featured = [0]*count_unique_word
    for word in x.split(' '):
        word_featured[dict_unique_words[word]] += 1
    train_featured.append(word_featured)

for x in x_test:
    word_featured = [0]*count_unique_word
    for word in x.split(' '):  
        if word in dict_unique_words:
            word_featured[dict_unique_words[word]] += 1
    test_featured.append(word_featured)

train_featured = np.array(train_featured)
test_featured = np.array(test_featured)


print(count_unique_word)
print(x_train[0])
print(sum(train_featured[0]))
print(x_train[1])
print(sum(train_featured[1]))


4194
crust good
2
tasty texture nasty
3


(e)Pick your postprocessing strategy. Since the vast majority of English words will not appear in
most of the reviews, most of the feature vector elements will be 0. This suggests that we need
a postprocessing or normalization strategy that combats the huge variance of the elements
in the feature vector. You may want to use one of the following strategies. Whatever choices
you make, explain why you made the decision.
• log-normalization. For each element of the feature vector x, transform it into f (x) Æ
log (x Å1).
• l1 normalization. Normalize the l1 normof the feature vector, ˆx Æ
x
j x j
.
• l2 normalization. Normalize the l2 normof the feature vector, ˆx Æ
x
kxk
.
• Standardize the data by subtracting the mean and dividing by the variance.

Explanation: l2 can provide a higher accuracy of the normalization so we choose to postprocess the data through L2 strategy.

In [156]:
#postprocessing
#from sklearn import preprocessing
#from sklearn.preprocessing import FunctionTransformer

#log-normalization. For each element of the feature vector x, transform it into f (x) Æ log (x Å1)
transformer = FunctionTransformer(np.log1p, validate=True)
train_featured_log = transformer.transform(train_featured)
test_featured_log = transformer.transform(test_featured)
print(train_featured_log)
print(test_featured_log)

#I1 normalization. Normalize the l1 normof the feature vector, ˆx Æ x j x j 
train_featured_l1 = preprocessing.normalize(train_featured, norm='l1')
test_featured_l1 = preprocessing.normalize(test_featured, norm='l1')
print(train_featured_l1)
print(test_featured_l1)

#l2 normalization. Normalize the l2 normof the feature vector, ˆx Æ x kxk 
train_featured_l2 = preprocessing.normalize(train_featured, norm='l2')
test_featured_l2 = preprocessing.normalize(test_featured, norm='l2')
print(train_featured_l2)
print(test_featured_l2)

#Standardize the data by subtracting the mean and dividing by the variance.
train_featured_scale = preprocessing.scale(train_featured)
test_featured_scale = preprocessing.scale(test_featured)
print(train_featured_scale)
print(test_featured_scale)



[[0.69314718 0.69314718 0.         ... 0.         0.         0.        ]
 [0.         0.         0.69314718 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.69314718 0.         ... 0.69314718 0.69314718 0.69314718]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0.5        0.5        0.         ... 0.         0.         0.        ]
 [0.         0.         0.33333333 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.0625     0.    

(f ) Sentiment prediction. Train a logistic regression model (you can use existing packages here)
on the training set and test on the testing set. Report the classification accuracy and confusion
matrix. Inspecting the weight vector of the logistic regression, what are the words that
play the most important roles in deciding the sentiment of the reviews? Repeat this with a
Naive Bayes classifier and compare performance.

In [193]:
#Sentiment prediction

#classfication accuracy
#Logistic Regression
L_R = LogisticRegression()
L_R.fit(train_featured_l2,y_train)
y_prediction_LR = L_R.predict(test_featured_l2)
print("Logistic Accuracy:",metrics.accuracy_score(y_test, y_prediction_LR))
#Confusion Matrix for Logistic Regression
cm_LR = confusion_matrix(y_test, y_prediction_LR)
print("Confusion matrix for logistic regression:\n", cm_LR)

#Naive Bayes classfier
Ber_NB = BernoulliNB()
Ber_NB.fit(train_featured_l2,y_train)
y_prediction_NB = Ber_NB.predict(test_featured_l2)
print("Bernoulli Accuracy:",metrics.accuracy_score(y_test, y_prediction_NB))
#Confusion Matrix for Naive Bayes
cm_NB = confusion_matrix(y_test, y_prediction_NB)
print("Confusion matrix for Naive Bayes:\n", cm_NB)

# the words that play the most important roles in deciding the sentiment of the reviews
def Most_Important_Word(m, d):
    freq = abs(m.coef_[0])
    most_important_word_idx = np.argsort(freq)[::-1][:10]
    re = []
    for word,value in d.items():
        if value in most_important_word_idx: 
            re.append(str(word))
    return re
print("Top 10 important word of Logistic Regression:", Most_Important_Word(L_R, dict_unique_words))
print("Top 10 important word of Naive Bayes:", Most_Important_Word(Ber_NB, dict_unique_words))

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Accuracy: 0.8066666666666666
Confusion matrix for logistic regression:
 [[253  47]
 [ 69 231]]
Bernoulli Accuracy: 0.8133333333333334
Confusion matrix for Naive Bayes:
 [[251  49]
 [ 63 237]]
Top 10 important word of logistic: ['good', 'worst', 'poor', 'bad', 'love', 'best', 'nice', 'great', 'delicious', 'excellent']
Top 10 important word of Naive Bayes: ['crust', 'chilly', 'unremarkable', 'author', 'livingworking', 'abstruse', 'culture', 'reenactment', 'emotionally', 'masculinity']


(g) N-gram model. Similar to the bag of words model, but now you build up a dictionary of ngrams,
which are contiguous sequences of words. For example, “Alice fell down the rabbit
hole” would then map to the 2-grams sequence: ["Alice fell", "fell down", "down the", "the
rabbit", "rabbit hole"], and all five of those symbols would be members of the n-gram dictionary.
Try n Æ 2, repeat (d)-(g) and report your results.

In [200]:
#N-gram model

n_gram_unique_word_dict = dict()
n_gram_unique_word_count = 0
n_gram_train_featured = [] 
n_gram_test_featured = []  

for n in x_train:
    words = n.split()
    for i in range(len(words)-2):
        sequence = ' '.join(words[i:i+2])
        if sequence not in n_gram_unique_word_dict:
            n_gram_unique_word_dict[sequence] = unique_word_count
            unique_word_count +=1

for n in x_train:
    word_featured = [0]*unique_word_count
    words = n.split()
    for i in range(len(words) - 2):
        sequence = ' '.join(words[i:i + 2])
        word_featured[n_gram_unique_word_dict[sequence]] += 1
    n_gram_train_featured.append(word_featured)
n_gram_train_featured_l2 = preprocessing.normalize(n_gram_train_featured, norm='l2')

for n in x_test:
    word_featured = [0]*unique_word_count
    words = n.split()
    for i in range(len(words) - 2):
        sequence = ' '.join(words[i:i + 2])
        if sequence in n_gram_unique_word_dict:
            word_featured[n_gram_unique_word_dict[sequence]] += 1
    n_gram_test_featured.append(word_featured)
    
n_gram_test_featured_l2 = preprocessing.normalize(n_gram_test_featured, norm='l2')

L_R = LogisticRegression()
L_R.fit(n_gram_train_featured_l2,y_train)
y_prediction_LR = L_R.predict(n_gram_test_featured_l2)
print("Logistic Accuracy:",metrics.accuracy_score(y_test, y_prediction_LR))
print("Top 10 important word of Logistic Regression:", find_important(L_R, n_gram_unique_word_dict))
cm_lr = confusion_matrix(y_test, y_prediction_LR)
print("Confusion matrix for Logistic Regression:\n", cm_lr)

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Accuracy: 0.64
Top 10 important word of Logistic Regression: ['is not', 'was not', 'waste your', 'and the', 'was very', 'would not', 'the worst', 'the best', 'a great', 'I love']
Confusion matrix for Logistic Regression:
 [[229  71]
 [145 155]]


(h) PCA for bag of words model. The features in the bag of words model have large redundancy.
Implement PCA to reduce the dimension of features calculated in (e) to 10, 50 and 100 respectively.
Using these lower-dimensional feature vectors and repeat (f ), (g). Report corresponding
clustering and classification results. (Note: You should implement PCA yourself,
but you can use numpy.svd or some other SVD package. Feel free to double-check your PCA
implementation against an existing one)

In [202]:
#PCA for bag of words model
#from sklearn.decomposition import PCA

#Implement PCA
def PCA(features,n):
    t = np.mean(features, 0) # mean of each column
    features = features - t
    eva, evs, eve = np.linalg.svd(features, full_matrices=False)
    e_ = eva[:, :n]*evs[:n]
    return e_

def lg(pca_x_train, pca_x_test, y_train, y_test):
    L_R = LogisticRegression()
    L_R.fit(pca_x_train,y_train)
    y_prediction_LR = L_R.predict(pca_x_test)
    return metrics.accuracy_score(y_test, y_prediction_LR)

def Implement_PCA(train_featured,test_featured, n):  
    pca_x_train = PCA(train_featured,n)
    pca_x_test = PCA(test_featured,n)
    accuracy = lg(pca_x_train, pca_x_test, y_train, y_test)
    print("dimension of features:", n, "accuracy after PCA:", accuracy)

#to reduce the dimension of features calculated in (e) to 10, 50 and 100 respectively.
print("PCA for bag of words:")
Implement_PCA(train_featured, test_featured, 10)
Implement_PCA(train_featured, test_featured, 50)
Implement_PCA(train_featured, test_featured, 100)
print("2-gram:")
Implement_PCA(n_gram_train_featured, n_gram_test_featured, 10)
Implement_PCA(n_gram_train_featured, n_gram_test_featured, 50)
Implement_PCA(n_gram_train_featured, n_gram_test_featured, 100)

PCA for bag of words:


//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


dimension of features: 10, accuracy after PCA: 0.35333333333333333


//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


dimension of features: 50, accuracy after PCA: 0.3466666666666667


...

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


dimension of features: 10, accuracy after PCA: 0.485


//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


dimension of features: 50, accuracy after PCA: 0.465
dimension of features: 100, accuracy after PCA: 0.46166666666666667


//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


As we can see from the result, when dimensions of features is 100, after pca, its accuracy is the highest.

In [205]:
#solve the logistic regression for dimensions of features = 100
from sklearn.decomposition import PCA
pca = PCA(n_components = 100)
pca_1 = pca.fit_transform(train_featured)
pca_2 = pca.fit_transform(test_featured)
L_R = LogisticRegression()
L_R.fit(pca_1,y_train)
y_prediction_LR_100 = L_R.predict(pca_2)
print("Logistic Accuracy:",metrics.accuracy_score(y_test, y_prediction_LR_100))
print(pca_1)

[[ 5.42417182e-02  8.65271223e-01  1.44121633e-01 ... -1.35898162e-02
  -1.26040918e-02 -1.04450488e-02]
 [ 1.03991568e-02 -4.80540250e-03 -1.27947749e-01 ...  3.85017940e-02
   3.69153521e-02 -2.14583840e-02]
 [ 6.70847491e-05 -6.67484864e-03 -1.35120194e-01 ... -1.17845867e-01
   9.49185506e-02  2.66480539e-02]
 ...
 [-4.55276367e-02  2.96946723e-03 -8.80614976e-02 ... -1.31945176e-01
  -7.07224558e-02 -7.24686186e-02]
 [-1.24948553e-01 -2.58973259e-02 -4.21827373e-02 ... -1.78451911e-02
   5.11616847e-02 -1.09037247e-01]
 [ 6.47968718e-02  9.24611277e-01  1.96114322e-01 ... -7.74640416e-02
  -2.05394148e-01  1.13887436e-01]]
Logistic Accuracy: 0.58


//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(i) Algorithms comparison and analysis. According to the above results, compare the performances
of bag of words, 2-gram and PCA for bag of words. Which method performs best in
the prediction task and why? What do you learn about the language that people use in online
reviews (e.g., expressions that will make the posts positive/negative)? Hint: Inspect the
clustering results and the weights learned from logistic regression.

From the above results, we can find that the bag of words method through Logistic model performs best in the prediction task. It might be becasue it will not reduce dimensions of features and preserve meaningful information such as words and sentences itself.
From the results of the top 10, we can find that most important words are positive emotional expressions such as "good", "delicious".